In [62]:
from bs4 import BeautifulSoup
from langdetect import detect
import nltk
import re
import os

In [2]:
def print_list(l):
    lenght=len(l)
    if lenght==0:
        return ""
    if lenght==1:
        return l[0]
    to_print=""
    if lenght>1:
        for i in range(lenght-1):
            to_print+=l[i]+", "
        to_print+=l[lenght-1]
    return to_print

In [50]:
class bookTitle():
    def name():
        return "bookTitle"
    def parse(soup):
        bookTitle = soup.find_all('h1')[0].contents[0]
        bookTitle = " ".join(bookTitle.split())
        return (bookTitle)

class bookSeries():
    def name():
        return "bookSeries"
    def parse(soup):
        bookSeries=""
        bookSeries=soup.find('h2',id="bookSeries").text.strip()[1:-1]
        return bookSeries

class bookAuthors():
    def name():
        return "bookAuthors"
    def parse(soup):
        bookAuthors=[]
        for element in soup.find_all("span",itemprop="name"):
            bookAuthors.append(element.text.strip())
        return print_list(bookAuthors)

class ratingValue():
    def name():
        return "ratingValue"
    def parse(soup):
        ratingValue = soup.find_all('span',itemprop="ratingValue")[0].contents[0].split('\n')[1].strip() 
        return ratingValue

class ratingCount():
    def name():
        return "ratingCount"  
    def parse(soup):
        return str(soup.find("meta",itemprop="ratingCount").get("content"))

class reviewCount():
    def name():
        return "reviewCount"
    def parse(soup):
        return str(soup.find("meta",itemprop="reviewCount").get("content"))


class Plot():
    def name():
        return "Plot"
    def parse(soup):
        def headingToRemove(Plot): 
            to_check=Plot.find("i")
            if to_check:
                forbidden_strings=["isbn","edition","librarian's note"]
                for string in forbidden_strings:
                    if string in to_check.text.lower():
                        Plot.find("i").decompose()

        Plot=soup.find("div", id="descriptionContainer").find_all("span")

        if len(Plot)==2:
            Plot=Plot[1]
            headingToRemove(Plot)
            Plot=Plot.text
            Plot=" ".join(Plot.split())
            Plot=Plot.replace("\\","")
        elif len(Plot)==1: 
            Plot=Plot[0]
            headingToRemove(Plot)
            Plot=Plot.text
            Plot=" ".join(Plot.split())
            Plot=Plot.replace("\\","")
        else:
            Plot=""
        return Plot


class NumberOfPages():
    def name():
        return "NumberOfPages"
    def parse(soup):
        N_pages=soup.find_all('span', itemprop="numberOfPages")
        if N_pages:
            return N_pages[0].contents[0].replace('\n', '').strip().split()[0]
        return ""

class Publishing_Date():
    def name():
        return "Publishing_Date"
    def parse(soup):
        elements = [e for e in soup.find_all("div", class_="row") if re.match(r'Published',e.text.strip())]
        #We first try to get the "first published date"
        if elements:
            date=re.findall(r'(?<=\(first published )(.*?)(?=\))',elements[0].text)
        else:
            return ""
        if date:
            return date[0]
        #We now see if there is a publishing date (but not a first publishing one).
        date=" ".join(elements[0].text.split()).split()
        #Handling the issue that not always the date is in the same format 
        if date[1]!="by":
            Publishing_Date=date[1]
            if len(date)>2 and date[2]!="by":
                Publishing_Date+=" "+date[2]
                if len(date)>3 and date[3]!="by":
                    Publishing_Date+=" "+date[3]
            return Publishing_Date
        else:
            return ""

        
class Characters():
    def name ():
        return "Characters"
    def parse(soup):
        Characters=soup.find_all("a",{'href': re.compile(r'^/characters/')})
        characters=[]
        for item in Characters:
            characters.append(" ".join(item.text.split()))
        return print_list(characters)

class Setting():
    def name():
        return "Setting"
    def parse(soup):
        Setting_temp=soup.find_all("div",class_="infoBoxRowItem")
        Setting=[]
        temp=[]
        Setting_places = []
        for element in Setting_temp:
            if element.find("a",{'href': re.compile(r'^/places/')}):
                Setting_places=element
        if Setting_places:
            temp=Setting_places.find_all()
        else:
            Setting=[]
        for element in temp:
            if element.name=="a":
                to_insert=element.text.split()
                Setting.append(" ".join(to_insert))
            if element.name=="span":
                to_add=element.text.split()
                Setting[-1]+=" "+(" ".join(to_add))
        #This is only a vert long workaround but seems to work
        for i in range(len(Setting)):
            Setting[i]=Setting[i].replace("…more","").replace("…less","").strip()
        Setting=list(dict.fromkeys([x for x in Setting if x]))
        return print_list(Setting)


class Url():
    def name():
        return "Url"
    def parse(soup):
        return re.findall(r'(?<=link href=")(.*?)(?=")',str(soup))[0]

In [57]:
#TEST
with open(name,"r",encoding="utf-8") as file:
    name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(3)+"\\"+str(3)+".html"
    soup=BeautifulSoup(file,features="html.parser")
Setting.parse(soup)

'Ajiro (Japan), Tokyo (Japan)'

In [ ]:
functions=[bookTitle,bookSeries,bookAuthors,ratingValue,ratingCount,reviewCount,Plot,NumberOfPages,Publishing_Date,\
           Characters,Setting,Url]

header = ""
for fun in functions[:-1]:
    header += fun.name()+"\t"
header +=functions[-1].name()
count = 1 #for the articles

for i in range(1,101):
    for j in range(1,301):
        name="C:\\Users\\Stefania\\ADM_HW3\\ADM3\\"+str(i)+"\\"+str(j)+".html"
        try:
            with open(name,"r",encoding="utf-8") as file:
                if os.path.getsize(name)<100000:
                    continue
                soup=BeautifulSoup(file,features="html.parser")
                second_line = ""
                for fun in functions[:-1]:
                    try:
                        second_line += fun.parse(soup)+"\t"
                    except:
                        print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i)
                        second_line += "" + "\t"
                try:
                    second_line += functions[-1].parse(soup)  
                except:
                    print("Unspecified exception in for function ", fun.name() ," for book j" , j, "in page i " , i) 
                    second_line += "" + "\t"
            name="C:\\Users\\Stefania\\ADM_HW3\\articles\\articles_"+str(count)+".tsv"
            with open(name,"w",encoding="utf-8") as file:
                file.write(header)
                file.write("\n")
                file.write(second_line)
            count +=1
        except FileNotFoundError:
            continue